In [3]:
# ---
# title: CLN Structured Note Simulator - Demo Notebook
# author: Rishabh Thole
# description: Demonstrates pricing and payoff simulation for a Credit-Linked Note (CLN)
# ---

# ## 1. Imports and setup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.params import DEFAULTS
from src.pricer import montecarlo_price, fair_coupon_for_target_yield
from src.reporter import save_results_to_excel, plot_payoff_hist
from src.simulator import correlated_default_sim

# ## 2. Load input basket
# Example: read from CSV in /data/
basket = pd.read_csv("/Users/rishabhthole/Desktop/Data/Projects/Structured Product Design & Pricing Automation/data/sample_basket.csv")
basket


ModuleNotFoundError: No module named 'src'

In [ ]:
# ## 3. Define parameters

names = basket["name"].tolist()
pd_annual = basket["annual_pd"].tolist()

correlation = 0.35
tenor = 2
num_sims = 50000
discount_rate = 0.03
recovery = 0.4

print(f"Running simulation for: {names}")


In [ ]:
# ## 4. Calibrate fair coupon

fair_c = fair_coupon_for_target_yield(
    target_yield=0.08,
    pd_annual=pd_annual,
    correlation=correlation,
    tenor=tenor,
    num_sims=num_sims,
    discount_rate=discount_rate,
    recovery=recovery
)
print(f"Fair coupon (approx.): {fair_c:.4%}")


In [ ]:
# ## 5. Run Monte Carlo pricing

pv_vals, _ = montecarlo_price(
    pd_annual=pd_annual,
    correlation=correlation,
    tenor=tenor,
    coupon=fair_c,
    num_sims=num_sims,
    discount_rate=discount_rate,
    recovery=recovery
)

print(f"Mean PV: {pv_vals.mean():.6f}  | Std: {pv_vals.std():.6f}")


In [ ]:
# ## 6. Visualize payoff distribution

plt.figure(figsize=(8, 5))
plt.hist(pv_vals, bins=60, color="steelblue")
plt.title("CLN Payoff Distribution (PV)")
plt.xlabel("Present Value")
plt.ylabel("Frequency")
plt.show()


In [ ]:
# ## 7. Save results to Excel

params = {
    "names": ",".join(names),
    "pd_annual": str(pd_annual),
    "correlation": correlation,
    "tenor": tenor,
    "num_sims": num_sims,
    "discount_rate": discount_rate,
    "recovery": recovery
}

save_results_to_excel(
    "../outputs/example_report.xlsx",
    params=params,
    price=pv_vals.mean(),
    std=pv_vals.std(),
    fair_coupon=fair_c
)

print("✅ Results saved to outputs/example_report.xlsx")


In [ ]:
# ## 8. Insights & extensions

print("""
Next steps you can explore:
1. Change correlation and see how fair coupon changes.
2. Replace PDs with actual credit spreads (convert to PD).
3. Add a yield curve for discounting (instead of flat rate).
4. Extend to t-copula or hybrid products.
""")
